<a href="https://colab.research.google.com/github/LarsAmker/ExplainGAN/blob/master/Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

In [2]:
# Fixes and checks I don't need anymore:
!pwd
#!mkdir ~/.kaggle
#!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
#!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
# Upload kaggle API key file.
# DON'T USE THIS - upload kaggle.json manually using the left-hand pane
uploaded = files.upload()

In [4]:
# Check what files and directories are in the current working directory. -a makes hidden ones (starting with .) visible
!ls -a

.  ..  abc  .config  .ipynb_checkpoints  .kaggle  kaggle.json  sample_data


In [5]:
# Download pneumonia data
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 99% 2.28G/2.29G [00:31<00:00, 85.2MB/s]
100% 2.29G/2.29G [00:31<00:00, 78.0MB/s]


In [0]:
! chmod 600 ~/.kaggle/kaggle.json